### Import

In [27]:
# !pip install scikit-learn
# !pip install lightgbm
# !pip install category_encoders

In [35]:
!pip install --upgrade lightgbm



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [28]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_selection import SelectFromModel
import lightgbm as lgb
from category_encoders import TargetEncoder
from sklearn.metrics import accuracy_score

### Data Load

In [29]:
train = pd.read_csv('train.csv').drop(columns=['ID'])
test = pd.read_csv('test.csv').drop(columns=['ID'])

In [30]:
X = train.drop('임신 성공 여부', axis=1)
y = train['임신 성공 여부']

### Data Pre-processing

In [31]:
categorical_columns = [
    "시술 시기 코드", "시술 당시 나이", "시술 유형", "특정 시술 유형", "배란 자극 여부",
    "배란 유도 유형", "단일 배아 이식 여부", "착상 전 유전 검사 사용 여부", "착상 전 유전 진단 사용 여부",
    "남성 주 불임 원인", "남성 부 불임 원인", "여성 주 불임 원인", "여성 부 불임 원인", "부부 주 불임 원인",
    "부부 부 불임 원인", "불명확 불임 원인", "불임 원인 - 난관 질환", "불임 원인 - 남성 요인",
    "불임 원인 - 배란 장애", "불임 원인 - 여성 요인", "불임 원인 - 자궁경부 문제",
    "불임 원인 - 자궁내막증", "불임 원인 - 정자 농도", "불임 원인 - 정자 면역학적 요인",
    "불임 원인 - 정자 운동성", "불임 원인 - 정자 형태", "배아 생성 주요 이유", "총 시술 횟수",
    "클리닉 내 총 시술 횟수", "IVF 시술 횟수", "DI 시술 횟수", "총 임신 횟수", "IVF 임신 횟수",
    "DI 임신 횟수", "총 출산 횟수", "IVF 출산 횟수", "DI 출산 횟수", "난자 출처", "정자 출처",
    "난자 기증자 나이", "정자 기증자 나이", "동결 배아 사용 여부", "신선 배아 사용 여부",
    "기증 배아 사용 여부", "대리모 여부", "PGD 시술 여부", "PGS 시술 여부"
]

In [32]:
encoder = TargetEncoder(cols=categorical_columns)
X_encoded = encoder.fit_transform(X, y)
test_encoded = encoder.transform(test)


In [33]:
X_train, X_val, y_train, y_val = train_test_split(X_encoded, y, test_size=0.2, random_state=42)


In [37]:
from lightgbm import early_stopping

model = lgb.LGBMClassifier(n_estimators=1000, learning_rate=0.01, max_depth=10, random_state=42)

# LightGBM에서 조기 종료를 위한 콜백 설정
callbacks = [early_stopping(stopping_rounds=100, verbose=True)]

model.fit(X_train, y_train, eval_set=[(X_val, y_val)], callbacks=callbacks)



[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 53102, number of negative: 151978
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.018894 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 779
[LightGBM] [Info] Number of data points in the train set: 205080, number of used features: 64
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258933 -> initscore=-1.051521
[LightGBM] [Info] Start training from score -1.051521
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[949]	valid_0's binary_logloss: 0.485906


LGBMClassifier(learning_rate=0.01, max_depth=10, n_estimators=1000,
               random_state=42)

In [38]:
y_pred = model.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print(f"검증 데이터 정확도: {accuracy:.4f}")

검증 데이터 정확도: 0.7486


### Train

In [39]:
feature_importances = model.feature_importances_
important_features = pd.Series(feature_importances, index=X.columns).sort_values(ascending=False)

print("\n상위 중요 특성:")
print(important_features.head(10))



상위 중요 특성:
시술 당시 나이                 2767
이식된 배아 수                 2069
배아 이식 경과일                1806
총 생성 배아 수                1720
수집된 신선 난자 수              1649
시술 시기 코드                 1446
저장된 배아 수                 1257
IVF 시술 횟수                1003
클리닉 내 총 시술 횟수             997
임신 시도 또는 마지막 임신 경과 연수     903
dtype: int32


### Predict

In [40]:
selector = SelectFromModel(model, prefit=True, threshold='median')
X_train_selected = selector.transform(X_train)
X_val_selected = selector.transform(X_val)

c:\Users\tjddl\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
c:\Users\tjddl\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


### Submission

In [42]:
from lightgbm import early_stopping, log_evaluation

# 모델 설정
final_model = lgb.LGBMClassifier(n_estimators=1000, learning_rate=0.01, max_depth=10, random_state=42)

# LightGBM의 콜백 설정
callbacks = [
    early_stopping(stopping_rounds=100, verbose=True),
    log_evaluation(10)  # 10번마다 로그 출력
]

# 모델 학습
final_model.fit(X_train_selected, y_train, eval_set=[(X_val_selected, y_val)], callbacks=callbacks)


c:\Users\tjddl\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Number of positive: 53102, number of negative: 151978
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.010101 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 630
[LightGBM] [Info] Number of data points in the train set: 205080, number of used features: 34
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258933 -> initscore=-1.051521
[LightGBM] [Info] Start training from score -1.051521
Training until validation scores don't improve for 100 rounds
[10]	valid_0's binary_logloss: 0.557659
[20]	valid_0's binary_logloss: 0.548339
[30]	valid_0's binary_logloss: 0.540488
[40]	valid_0's binary_logloss: 0.533832
[50]	valid_0's binary_logloss: 0.528117
[60]	valid_0's binary_logloss: 0.523202
[70]	valid_0's binary_logloss: 0.518968
[80]	valid_0's binary_logloss: 0.515291
[90]	valid_0's binary_logloss: 0.512077
[100]	valid_0's bin

LGBMClassifier(learning_rate=0.01, max_depth=10, n_estimators=1000,
               random_state=42)

In [43]:
final_y_pred = final_model.predict(X_val_selected)
final_accuracy = accuracy_score(y_val, final_y_pred)
print(f"중요 특성 기반 검증 데이터 정확도: {final_accuracy:.4f}")


c:\Users\tjddl\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


중요 특성 기반 검증 데이터 정확도: 0.7483


In [44]:
test_selected = selector.transform(test_encoded)
test_predictions = final_model.predict(test_selected)


c:\Users\tjddl\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
c:\Users\tjddl\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [45]:
submission = pd.DataFrame({'ID': pd.read_csv('test.csv')['ID'], '임신 성공 여부': test_predictions})
submission.to_csv('submission.csv', index=False)
print("최종 예측 결과가 'submission.csv'로 저장되었습니다.")

최종 예측 결과가 'submission.csv'로 저장되었습니다.
